In [106]:
import string
import glob
import tqdm
from PIL import Image

imgpath = "/content/drive/MyDrive/samples/*.png"
imgfiles = glob.glob(imgpath)
print(imgfiles)
print(len(imgfiles))

['/content/drive/MyDrive/samples/28348.png', '/content/drive/MyDrive/samples/28x47.png', '/content/drive/MyDrive/samples/268g2.png', '/content/drive/MyDrive/samples/264m5.png', '/content/drive/MyDrive/samples/25w53.png', '/content/drive/MyDrive/samples/25257.png', '/content/drive/MyDrive/samples/25m6p.png', '/content/drive/MyDrive/samples/25p2m.png', '/content/drive/MyDrive/samples/25egp.png', '/content/drive/MyDrive/samples/253dc.png', '/content/drive/MyDrive/samples/24f6w.png', '/content/drive/MyDrive/samples/24pew.png', '/content/drive/MyDrive/samples/245y5.png', '/content/drive/MyDrive/samples/244e2.png', '/content/drive/MyDrive/samples/23n88.png', '/content/drive/MyDrive/samples/23mdg.png', '/content/drive/MyDrive/samples/2356g.png', '/content/drive/MyDrive/samples/22d5n.png', '/content/drive/MyDrive/samples/243mm.png', '/content/drive/MyDrive/samples/226md.png', '/content/drive/MyDrive/samples/2cg58.png', '/content/drive/MyDrive/samples/2cgyx.png', '/content/drive/MyDrive/samples

In [87]:
import torch
import torch.nn as nn
import torch.nn.functional as F

#기본 블럭
class BasicBlock(nn.Module):
  def __init__(self, 
               in_channels, 
               out_channels, 
               kernel_size=(3, 5), 
               stride=(2, 1)):
    
    super(BasicBlock, self).__init__()

    self.c1 = nn.Conv2d(in_channels=in_channels, 
                        out_channels=out_channels,
                        kernel_size=kernel_size, 
                        stride=stride)
      
    self.c2 = nn.Conv2d(in_channels=out_channels, 
                        out_channels=out_channels,
                        kernel_size=(3, 3), padding=1)

    self.downsample = nn.Conv2d(in_channels=in_channels, 
                                out_channels=out_channels,
                                kernel_size=kernel_size, 
                                stride=stride)

    self.bn1 = nn.BatchNorm2d(num_features=out_channels)
    self.bn2 = nn.BatchNorm2d(num_features=out_channels)

    self.relu = nn.ReLU()

  def forward(self, x):
    x_ = x

    x = self.c1(x)
    x = self.bn1(x)
    x = self.relu(x)
    x = self.c2(x)
    x = self.bn2(x)

    x_ = self.downsample(x_)

    x += x_
    x = self.relu(x)

    return x

In [88]:
class CRNN(nn.Module):
  def __init__(self, output_size):
    super(CRNN, self).__init__()

    self.c1 = BasicBlock(in_channels=3, out_channels=64)
    self.c2 = BasicBlock(in_channels=64, out_channels=64)
    self.c3 = BasicBlock(in_channels=64, out_channels=64)
    self.c4 = BasicBlock(in_channels=64, out_channels=64)
    self.c5 = nn.Conv2d(64, 64, kernel_size=(2, 5))

    self.gru = nn.GRU(64, 64, batch_first=False)

    self.fc1 = nn.Linear(in_features=64, out_features=128)
    self.fc2 = nn.Linear(in_features=128, out_features=output_size)
    self.relu = nn.ReLU()

  def forward(self, x):
    x = self.c1(x)
    x = self.c2(x)
    x = self.c3(x)
    x = self.c4(x)
    x = self.c5(x)

    x = x.view(x.shape[0], 64, -1)
    x = x.permute(2, 0, 1)

    x, _ = self.gru(x)

    x = self.fc1(x)
    x = self.relu(x)
    x = self.fc2(x)

    x = F.log_softmax(x, dim=-1)

    return x

In [129]:
import numpy as np
from torch.utils.data.dataset import Dataset

def get_BOW(corpus):
  BOW = {"<pad>":0}

  for letter in corpus:
      if letter not in BOW.keys():
        BOW[letter] = len(BOW.keys())

  return BOW

class Captcha(Dataset):
  def __init__(self, imgpath, train=True):
    self.corpus = string.ascii_lowercase + string.digits
    self.BOW = get_BOW(self.corpus)

    self.imgfiles = imgfiles

    self.train = train
    self.trainset = self.imgfiles[:int(len(self.imgfiles)*0.8)]
    self.testset = self.imgfiles[int(len(self.imgfiles)*0.8):]

  def get_seq(self, line):
    label = []

    print(line)

    for letter in line:
      label.append(self.BOW[letter])
      return label

  def __len__(self):
    if self.train:
      return len(self.imgfiles)
    else:
      return len(self.testset)

  def __getitem__(self, idx):
    if self.train:
      data = Image.open(self.trainset[idx]).convert("RGB")
      label = self.trainset[idx].split("/")[-1]
      label = label.split(".png")[0]
      label = self.get_seq(label)
      data = np.array(data).astype(np.float32)
      data = np.transpose(data, (2, 0, 1))
      label = np.array(label)
      return data, label

    else:
      data = Image.open(self.testset[idx]).convert("RGB")
      label = self.testset[idx].split("/")[-1]
      label = label.split(".png")[0]
      label = self.get_seq(label)
      
      data = np.array(data).astype(np.float32)
      label = np.array(label)

      return data, label

from torch.utils.data.dataloader import DataLoader
from torch.optim.adam import Adam
import tqdm

device = "cuda" if torch.cuda.is_available() else "cpu"

dataset = Captcha(imgpath)
loader = DataLoader(dataset, batch_size=8)

model = CRNN(output_size=len(dataset.BOW)).to(device)
optim = Adam(model.parameters(), lr=0.0001)

for epoch in range(200):
  iterator = tqdm.tqdm(loader)

  for data, label in iterator:
    optim.zero_grad()
    preds = model(data.to(device))

    preds_size = torch.IntTensor([preds.size(0)] * 8).to(device)
    target_len = torch.IntTensor([len(txt) for txt in label]).to(device)
      
    loss = nn.CTCLoss(blank=0)(preds, label.to(device), preds_size, target_len)

    loss.backward()
    optim.step()

    iterator.set_description(f"epoch{epoch+1} loss:{loss.item()}")

torch.save(model.state_dict(), "CRNN.pth")

  0%|          | 0/130 [00:00<?, ?it/s]

28348
28x47
268g2
264m5
25w53
25257
25m6p
25p2m


epoch1 loss:651.2728271484375:   1%|          | 1/130 [00:00<02:02,  1.05it/s]

25egp
253dc
24f6w
24pew
245y5
244e2
23n88
23mdg


epoch1 loss:606.80078125:   2%|▏         | 2/130 [00:01<02:02,  1.04it/s]

...

nf8b8
nf2n8
nf7bn
neggn
neecd
ndecc
ndme7
ndyfe


epoch1 loss:115.50137329101562:  79%|███████▉  | 103/130 [01:43<00:26,  1.03it/s]

ne325
ndg2b
pdw38
pcpg6
pdcp4
pcede
pcm7f
p7fyp


epoch1 loss:113.44090270996094:  80%|████████  | 104/130 [01:44<00:26,  1.00s/it]


IndexError: ignored